In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF

# Machine learning
import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import os
from datetime import datetime

# MSE imports
import kepler_sieve
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts
from asteroid_model import make_model_ast_pos
from nearest_asteroid import nearest_ast_elt, calc_elt_pos, ts
from nearest_asteroid import nearest_ast_elt_cov, plot_elt_transform_pdf, plot_elt_transform_map
from nearest_asteroid import ast_elt_transform, calc_beta, make_interp_x
from astro_utils import datetime_to_mjd

Found 4 GPUs.  Setting memory growth = True.


In [2]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

In [3]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [4]:
# Review orbital elements of known asteroids
ast_elt

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


## Search for Asteroid with Nearest Elements Based on Orbital Trajectories

In [5]:
# First 64 odd numbered asteroids
batch_size = 64
ast_nums = np.arange(1,2*batch_size,2, dtype=np.int32)

# Candidate elements from these asteroids_
elts_ast = asteroid_elts(ast_nums)

In [6]:
# Manually change element_id to disambiguate concepts of element_id and asteroid_num
elts_ast.element_id = np.arange(batch_size)

In [7]:
# Review candidate elements
elts_ast

,element_id,a,e,inc,Omega,omega,f,epoch
0,0,2.769165,0.076009,0.184901,1.401596,1.284522,1.501306,58600.0
1,1,2.669150,0.256942,0.226699,2.964490,4.330836,0.996719,58600.0
2,2,2.574249,0.191095,0.093672,2.470978,6.260280,-1.738676,58600.0
3,3,2.385334,0.231206,0.096406,4.530233,2.535354,-3.593901,58600.0
4,4,2.385637,0.123114,0.097334,1.202682,0.112004,-1.697553,58600.0
...,...,...,...,...,...,...,...,...
59,59,2.581258,0.080455,0.100988,3.554433,2.990562,-0.184508,58600.0
60,60,3.451839,0.131877,0.132604,1.276228,5.200234,3.674133,58600.0
61,61,2.694898,0.120490,0.111943,5.370912,2.195213,-0.875056,58600.0
62,62,2.744362,0.077776,0.081391,2.949472,1.916123,-3.376722,58600.0


In [8]:
# Search for near elements on unperturbed asteroids
elts_ast_near = nearest_ast_elt(elts_ast)

In [9]:
# Review nearest asteroid to these elements
elts_ast_near

,Num,Name,dist,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,0.0,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
3,3,Juno,0.0,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
5,5,Astraea,0.0,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
7,7,Iris,0.0,58600.0,2.385334,0.231206,0.096406,4.530233,2.535354,2.450785,5.51,0.15,JPL 109,-3.593901,1345.619184,0.004669,3.233187,-2.811498,0.782402,-524.864951
9,9,Metis,0.0,58600.0,2.385637,0.123114,0.097334,1.202682,0.112004,4.832147,6.28,0.17,JPL 114,-1.697553,1345.875332,0.004668,-0.136353,-0.382867,1.314686,310.816364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,Althaea,0.0,58600.0,2.581258,0.080455,0.100988,3.554433,2.990562,6.126533,8.42,0.15,JPL 74,-0.184508,1514.766179,0.004148,0.105157,0.077301,0.261810,37.766253
121,121,Hermione,0.0,58600.0,3.451839,0.131877,0.132604,1.276228,5.200234,3.820303,7.31,0.15,JPL 101,3.674133,2342.471004,0.002682,-2.269605,3.867410,0.193277,918.201500
123,123,Brunhild,0.0,58600.0,2.694898,0.120490,0.111943,5.370912,2.195213,5.582660,8.90,0.15,JPL 32,-0.875056,1615.890130,0.003888,0.582415,0.407884,1.282940,180.158914


In [10]:
# Review elts_ast, now with nearest asteroid information added
elts_ast

,element_id,a,e,inc,Omega,omega,f,epoch,nearest_ast_num,nearest_ast_dist
0,0,2.769165,0.076009,0.184901,1.401596,1.284522,1.501306,58600.0,1,0.0
1,1,2.669150,0.256942,0.226699,2.964490,4.330836,0.996719,58600.0,3,0.0
2,2,2.574249,0.191095,0.093672,2.470978,6.260280,-1.738676,58600.0,5,0.0
3,3,2.385334,0.231206,0.096406,4.530233,2.535354,-3.593901,58600.0,7,0.0
4,4,2.385637,0.123114,0.097334,1.202682,0.112004,-1.697553,58600.0,9,0.0
...,...,...,...,...,...,...,...,...,...,...
59,59,2.581258,0.080455,0.100988,3.554433,2.990562,-0.184508,58600.0,119,0.0
60,60,3.451839,0.131877,0.132604,1.276228,5.200234,3.674133,58600.0,121,0.0
61,61,2.694898,0.120490,0.111943,5.370912,2.195213,-0.875056,58600.0,123,0.0
62,62,2.744362,0.077776,0.081391,2.949472,1.916123,-3.376722,58600.0,125,0.0


## Search Around Perturbed Elements

In [11]:
# Inputs to perturb elements: large
sigma_a = 0.05
sigma_e = 0.01
sigma_inc_deg = 0.25
sigma_f_deg = 1.0
sigma_Omega_deg = 1.0
sigma_omega_deg = 1.0
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert= perturb_elts(elts_ast, sigma_a=sigma_a, sigma_e=sigma_e, 
                    sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                    sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                    mask_pert=mask_pert, random_seed=random_seed)

In [12]:
# Review perturbed elements
elts_pert

,element_id,a,e,inc,Omega,omega,f,epoch,nearest_ast_num,nearest_ast_dist
0,0,2.838801,0.076629,0.185336,1.423708,1.286215,1.505043,58600.0,1,0.0
1,1,2.650761,0.260451,0.224503,2.952139,4.341224,0.974977,58600.0,3,0.0
2,2,2.658979,0.190957,0.086906,2.478724,6.245999,-1.735653,58600.0,5,0.0
3,3,2.574076,0.233538,0.096705,4.543753,2.571874,-3.587175,58600.0,7,0.0
4,4,2.357869,0.123560,0.092699,1.186504,0.094446,-1.712979,58600.0,9,0.0
...,...,...,...,...,...,...,...,...,...,...
59,59,2.710286,0.079335,0.100670,3.554800,2.976454,-0.168489,58600.0,119,0.0
60,60,3.370120,0.132654,0.128909,1.288130,5.185578,3.711172,58600.0,121,0.0
61,61,2.669997,0.123158,0.105333,5.365497,2.184752,-0.857036,58600.0,123,0.0
62,62,2.596675,0.077010,0.079443,2.955129,1.879054,-3.403240,58600.0,125,0.0


In [13]:
# Search for near elements on unperturbed asteroids
elts_pert_near = nearest_ast_elt(elts_pert)

In [15]:
# Review elts_pert, now with nearest asteroid information added
elts_pert

,element_id,a,e,inc,Omega,omega,f,epoch,nearest_ast_num,nearest_ast_dist
0,0,2.838801,0.076629,0.185336,1.423708,1.286215,1.505043,58600.0,1141687,0.002623
1,1,2.650761,0.260451,0.224503,2.952139,4.341224,0.974977,58600.0,3,0.004228
2,2,2.658979,0.190957,0.086906,2.478724,6.245999,-1.735653,58600.0,417755,0.003576
3,3,2.574076,0.233538,0.096705,4.543753,2.571874,-3.587175,58600.0,529459,0.003443
4,4,2.357869,0.123560,0.092699,1.186504,0.094446,-1.712979,58600.0,234585,0.002984
...,...,...,...,...,...,...,...,...,...,...
59,59,2.710286,0.079335,0.100670,3.554800,2.976454,-0.168489,58600.0,1156228,0.003300
60,60,3.370120,0.132654,0.128909,1.288130,5.185578,3.711172,58600.0,1149232,0.009668
61,61,2.669997,0.123158,0.105333,5.365497,2.184752,-0.857036,58600.0,1205775,0.003321
62,62,2.596675,0.077010,0.079443,2.955129,1.879054,-3.403240,58600.0,418270,0.003645


## Build and Plot Transformed Orbital Elements

In [ ]:
# Set number of sample points
N_samp_u: int = 2**16
z_range: float = 6.0
N_samp_z: int = int(200*z_range + 1)

# Sample CDF levels: N_samp evenly spaced
cdf_samp_u = (np.arange(N_samp_u) + 0.5) / N_samp_u

# Sample CDF levels: z_samp points evenly spaced by Z
z_samp = np.linspace(-z_range, z_range, N_samp_z)
cdf_samp_z = norm.cdf(z_samp)
pdf_samp_z = norm.pdf(z_samp)

# Combine the two sets of sample points
cdf_samp = np.unique(np.hstack([cdf_samp_u, cdf_samp_z]))

In [ ]:
# Build transformed elements and interpolators
ast_elt_xf, interp_tbl = ast_elt_transform(ast_elt)

In [ ]:
# Review orbital elements with extra transformed columns
ast_elt_xf

## Semimajor Axis, a

In [ ]:
# Plot PDF of a
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='log_a')

In [ ]:
# Plot transform of a
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='log_a')

## Eccentricity, e

In [ ]:
# Plot PDF of e
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='e')

In [ ]:
# Plot transform of e
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='e')

## Inclination, inc

In [ ]:
# Plot PDF of sin_inc
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

In [ ]:
# Plot transform of sin_inc
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_inc')

## Omega

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

In [ ]:
# Plot PDF of cos_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_Omega')

## omega

In [ ]:
# Plot PDF of sin_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_omega')

In [ ]:
# Plot PDF of cos_omega
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

In [ ]:
# Plot PDF of sin_Omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='cos_omega')

## True Anomaly, f

In [ ]:
# Plot PDF of sin_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot map of sin_omega
plot_elt_transform_map(ast_elt_xf=ast_elt_xf, elt_name='sin_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

In [ ]:
# Plot PDF of cos_f
plot_elt_transform_pdf(ast_elt_xf=ast_elt_xf, elt_name='cos_f')

## Covariance of Transformed Elements

In [ ]:
# Build tranformation matrix beta and X_beta for computing distance to orbital elements
beta, X_beta = calc_beta(ast_elt_xf)

In [ ]:
# Assemble covaraiance matrix
Q = np.cov(X_beta, rowvar=False)

In [ ]:
# Demonstrate that Q is the 9x9 identity matrix
Q_err = np.linalg.norm(Q - np.identity(9))
print(f'Error of matrix Q vs. identity: {Q_err:6.2e}')

## Search for Asteroid with Nearest Orbital Elements Based on Covariance of Transformed Elements

In [ ]:
# First 32 odd numbered asteroids (don't want consecutive to disambiguate rows and columns)
elt = ast_elt.iloc[0:64:2].copy()

In [ ]:
# Search for near elements on unperturbed asteroids
ast_elt_near = nearest_ast_elt(elt)

In [ ]:
# Review near elements - should be exact matches with 0 distance
ast_elt_near.iloc[0:10]

In [ ]:
# Inputs to perturb elements: large
sigma_a = 0.01
sigma_e = 0.002
sigma_inc_deg = 0.25
sigma_f_deg = 0.5
sigma_Omega_deg = 0.5
sigma_omega_deg = 0.5
mask_pert = None
random_seed = 42

# Perturb orbital elements
elts_pert = perturb_elts(elt, sigma_a=sigma_a, sigma_e=sigma_e, 
                    sigma_inc_deg=sigma_inc_deg, sigma_f_deg=sigma_f_deg, 
                    sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                    mask_pert=mask_pert, random_seed=random_seed)

In [ ]:
# Search for near elements on perturbed asteroids
ast_elt_near_pert = nearest_ast_elt(elts_pert)

In [ ]:
# Review near elements - should be close but not zero
ast_elt_near_pert

## Search for Asteroid with Nearest Elements Based on Orbital Trajectories

In [ ]:
# Set time array
t0 = datetime_to_mjd(datetime(2018,6,30))
t1 = datetime_to_mjd(datetime(2020,3,31))
dt = 7
ts = np.arange(t0, t1, dt)

In [ ]:
elt_ast = ast_elt.iloc[0:64]

In [ ]:
q_ast = calc_elt_pos(elt=elt_ast, ts=ts)

In [ ]:
q_ast.shape

In [ ]:
N_ast = ast_elt.shape[0]

In [ ]:
N_elt = elt.shape[0]

In [ ]:
N_t = ts.size

In [ ]:
(N_ast, N_elt, N_t, space_dims)

In [ ]:
# Load the known asteroid positions
q_ast = load_known_ast_pos()

In [ ]:
# Convert to a float32 tensor
X = tf.constant(q_ast, dtype=tf.float32)

In [ ]:
# Calculate position of element
q_elt = calc_elt_pos(elt, ts)
# Convert to a tensor; use float32 to save memory
Y = tf.constant(q_elt, dtype=tf.float32)

In [ ]:
# Number of asteroids and test elements
N_ast = X.shape[0]
N_elt = Y.shape[0]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
# Need to do this one candidate at a time b/c run out of memory when doing all at once
dist_i = tf.zeros(N_ast)

# Iterate over candidates
# for idx in range(N_elt):
idx = 0
dist_i = tf.reduce_mean(tf.linalg.norm(X - Y[idx], axis=-1), axis=-1)

In [ ]:
dist_i

In [ ]:
dist_i = tf.linalg.norm(X - Y[idx], axis=(-2, -1)) / np.sqrt(N_t)

In [ ]:
tf.argmin(dist_i)

In [ ]:
sz = 1
tf.linalg.norm(tf.reshape(X, (N_ast, 1, N_t, 3)) - tf.reshape(Y[idx:idx+sz], (1, sz, N_t, 3)), axis=(-2, -1)) / np.sqrt(N_t)

In [ ]:
# Col number of nearest asteroid elements
col_idx = np.argmin(dist, axis=0)

In [ ]:
col_idx

In [ ]:
dist.shape

In [ ]:
# Distance to nearest asteroid element
row_idx = np.arange(row_idx.size, dtype=np.int32)
dist_au = dist[row_idx, col_idx]

In [ ]:
def nearest_ast_elt_cart(elt):
    """
    Search for nearest asteroid element based on Cartesian orbits
    """
    # Calculate position of element
    q_elt = calc_elt_pos(elt, ts)
    # Convert to a tensor; use float32 to save memory
    Y = tf.constant(q_elt, dtype=tf.float32)

    # Number of asteroids and test elements
    N_ast = X.shape[0]
    N_elt = Y.shape[0]
    # Sqrt of N_ast used to get RMS distance
    sqrt_N_ast = np.sqrt(N_ast).astype(np.float32)

    # Need to do this one candidate at a time b/c run out of memory when doing all at once
    idx = np.zeros(N_elt, dtype=np.int32)
    dist = np.zeros(N_elt, dtype=np.float32)

    # Iterate over candidates    
    for i in range(N_elt):
        # dist_i is the distance between element i and all N_ast asteroids; shape [N_ast,]
        dist_i = tf.linalg.norm(X - Y[i], axis=(-2, -1))
        # The index of the closest asteroid
        idx[i] = tf.argmin(dist_i)
        # Save the index of the closest asteroid and its distance
        dist[i] = dist_i[idx[i]] / sqrt_N_ast

    # The closest asteroid elements
    ast_elt_nearest = ast_elt.iloc[idx].copy()
    
    # Add two extra columns showing the asteroid number and distance
    ast_elt_nearest.insert(loc=2, column='asteroid_num', value=ast_elt_nearest.Num)
    ast_elt_nearest.insert(loc=2, column='dist', value=dist)
    
    return ast_elt_nearest

In [ ]:
nearest_ast_elt_cart(elt)